In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [12]:
from sk import predict
from glob import glob
from tqdm import tqdm
import IPython.display as ipd

In [3]:
import malaya
import malaya_speech
import tensorflow as tf
malaya.__version__, tf.__version__

2021-10-08 00:11:05.563641: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/home/ubuntu/.local/lib/python3.8/site-packages/malaya_boilerplate/frozen_graph.py:24: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  warnings.warn(


('4.7', '2.5.0')

In [4]:
# !pip3 install tensorflow-text==2.5 --no-deps
# !pip3 install tensorflow_hub

In [5]:
# super_super_tiny = malaya.true_case.transformer(model = 'super-tiny-t5', device = 'cpu:0')

In [6]:
aligner = malaya_speech.force_alignment.deep_aligner(model = 'conformer-transducer', device = 'cpu:0')

2021-10-08 00:11:13.610826: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-08 00:11:13.612308: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-10-08 00:11:13.646486: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-08 00:11:13.646546: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2021-10-08 00:11:13.646556: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2021-10-08 00:11:13.646709: I tensorflow/stream_executor/cuda/cuda_diagnos

In [7]:
wavs = glob('salina-short/output-wav-salina/*.wav')
len(wavs)

26903

In [8]:
def put(alignment, length, min_threshold = 0.5, min_threshold_fullstop = 1.0):
    r = []
    for no, row in enumerate(alignment):
        if no > 0:
            if alignment[no]['start'] - alignment[no-1]['end'] >= min_threshold:
                r.append(',')

        r.append(row['text'])
    if length - alignment[-1]['end'] >= min_threshold_fullstop:
        r.append('.')
    else:
        r.append(',')
    return r

In [17]:
results = {}
for wav in tqdm(wavs):
    with open(wav.replace('-wav-', '-text-') + '.txt') as fopen:
        data = fopen.read().lower()
    y, sr = malaya_speech.load(wav)
    result = aligner.predict(y, data)
    if ' '.join([r['text'] for r in result['words_alignment']]) != data:
        continue
    o = put(result['words_alignment'], len(y) / sr)
    results[wav] = ' '.join(o)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26903/26903 [56:41<00:00,  7.91it/s]


In [18]:
len(results), len(wavs)

(25986, 26903)

In [19]:
results

{'salina-short/output-wav-salina/dua-puluh-tiga-24.mp3-177.wav': 'yang bertampal-tampal itu dilihatnya lagi , ada cicak berkejaran ,',
 'salina-short/output-wav-salina/dua-puluh-satu-22.mp3-571.wav': 'tetapi ,',
 'salina-short/output-wav-salina/empat-5.mp3-14.wav': 'botol botol minyak , botol botol ubat ,',
 'salina-short/output-wav-salina/tiga-4.mp3-741.wav': 'satu penarikan ,',
 'salina-short/output-wav-salina/sembilan-10.mp3-796.wav': 'dia tersedu-sedu terus sahaja ,',
 'salina-short/output-wav-salina/empat-belas-15.mp3-109.wav': 'kita bukannya tinggal di rumah betul ,',
 'salina-short/output-wav-salina/dua-puluh-lima-26.mp3-700.wav': 'sekarang tak ada takutlah , sebab group bayar ada ,',
 'salina-short/output-wav-salina/satu-2.mp3-390.wav': 'jawab katijah menerangkan ,',
 'salina-short/output-wav-salina/lapan-9.mp3-265.wav': 'dah habis sudah selama 2 tahun ,',
 'salina-short/output-wav-salina/dua-puluh-lima-26.mp3-81.wav': 'kerana puasa sudah berlalu ,',
 'salina-short/output-wav-s

In [20]:
ipd.Audio('salina-short/output-wav-salina/tujuh-belas-18.mp3-481.wav')

In [21]:
import json
with open('comma-fullstop-salina-short.json', 'w') as fopen:
    json.dump(results, fopen)